<h1 style="margin:0; line-height:1; padding:0;">Intro to Computational Thinking and Programming - Group 4</h1>
<h3 style="margin:0; line-height:1; padding:0; font-weight:normal;">Team Allocation Simulator</h3>


In [2]:
import csv
import matplotlib.pyplot as plt
import json
path = "records.csv" #this is the path to the data file
teamsize = 5

This block imports the necessary libraries, provides the path to the data file
and defines a variable for the teamsize to allow for further customisation

In [3]:
def tut_grp_json(path: str,json_filename: str='tut_grp'): #path: takes in filepath of csv file; json_filename: specify the file name of the json file without the filetype
    json_filename=json_filename+'.json'
    data={} #overall data
    with open(path,'r',newline='') as fObj:
        reader=csv.DictReader(fObj)
        for row in reader:
            key=row.pop('Tutorial Group') #remove the tutorial group from the row and store the variable
            if key not in data:
                data[key]=[row] #the value of data[key] is a list; if key not in, create new key
            else:
                data[key].append(row) #if key already exist, append to the existing list corresponding to the key
        ###sorting out the keys to make sure they are in order
        key_list=[]
        for key,student_info in data.items():
            key_list.append(int(key.split('-')[1])) #split string at '-'.
            key_list.sort() #sort from 1-120 in numerical order because python does not sort 'G-xx' in numerical order
        def myFunc(i):
            return 'G-'+str(i)
        key_list=list(map(myFunc,key_list))
        student_data = {key: data[key] for key in key_list}

        ###writing the the student_data object into a json file
        with open(json_filename,'w') as fObj:
            #json file cannot be modifiable
            json.dump(student_data,fObj,indent=2) #write to json file so the sorted dictionary cannot be accidentally messed up
        return json_filename


a .json file is because it is not in a format such that it can be modified, therefore this will prevent the json formatted dictionary from being accidentally mutated later on in the code.

In [4]:
def load_json_to_dict(json_file: str): #loads json file to a dictionary and returns a dictionary
    with open(json_file,'r') as jFileObj:
        values=json.load(jFileObj)
    return values

we can now use the above functions tut_grp_json load_json_to_dict to store the json file to our current file directory and to load the json file to a dictionary object

In [5]:
json_file=tut_grp_json(path)

values=load_json_to_dict(json_file)